In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
import pickle, datetime
import Preprocess as pp

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import  PIL.Image

## Data Loading

In [5]:
x_train, y_train, x_test, y_test, N_CATEGORY = pp.load_cat_dog(227, 227, 0.1, 0.1)
# x_train, y_train, x_test, y_test, N_CATEGORY = pp.load_cifar101(227, 227, 0.1, 0.1)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, N_CATEGORY)

(1735, 227, 227, 3) (6942, 227, 227, 3)


## AlexNet Architecture 

In [6]:
NB_CLASS = 1         # number of classes
LEARNING_RATE = 0.01
MOMENTUM = 0.9
ALPHA = 0.0001
BETA = 0.75
GAMMA = 0.1
DROPOUT = 0.5
WEIGHT_DECAY = 0.0005
LRN2D_norm = True       # whether to use batch normalization
DIM_ORDERING = 'tf'

def conv2D_lrn2d(x, nb_filter, nb_row, nb_col,
                 border_mode='same', subsample=(1, 1),
                 activation='relu', LRN2D_norm=True,
                 weight_decay=WEIGHT_DECAY, dim_ordering=DIM_ORDERING):
    '''
        Info:
            Function taken from the Inceptionv3.py script keras github
            Utility function to apply to a tensor a module Convolution + lrn2d
            with optional weight decay (L2 weight regularization).
    '''
    if weight_decay:
        W_regularizer = regularizers.l2(weight_decay)
        b_regularizer = regularizers.l2(weight_decay)
    else:
        W_regularizer = None
        b_regularizer = None

    x = Convolution2D(nb_filter, nb_row, nb_col,
                      subsample=subsample,
                      activation=activation,
                      border_mode=border_mode,
                      W_regularizer=W_regularizer,
                      b_regularizer=b_regularizer,
                      bias=False)(x)
    x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)

    if LRN2D_norm:

        x = LRN2D(alpha=ALPHA, beta=BETA)(x)
        x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)

    return x

In [11]:
INP_SHAPE = (227, 227, 3)  # 3 - Number of RGB Colours
img_input = Input(shape=INP_SHAPE)
CONCAT_AXIS = 3

# Channel 1 - Convolution Net Layer 1
x = conv2D_lrn2d(img_input, 3, 11, 11, subsample=(1, 1))
x = MaxPooling2D(strides=(4, 4), pool_size=(4, 4))(x)
x = ZeroPadding2D(padding=(1, 1))(x)

# Channel 1 - Convolution Net Layer 2
x = conv2D_lrn2d(x, 48, 55, 55, subsample=(1, 1))
x = MaxPooling2D(strides=(2, 2), pool_size=(2, 2))(x)
x = ZeroPadding2D(padding=(1, 1))(x)

# Channel 1 - Convolution Net Layer 3
x = conv2D_lrn2d(x, 128, 27, 27, subsample=(1, 1))
x = MaxPooling2D(strides=(2, 2), pool_size=(2, 2))(x)
x = ZeroPadding2D(padding=(1, 1))(x)

# Channel 1 - Convolution Net Layer 4
x = conv2D_lrn2d(x, 192, 13, 13, subsample=(1, 1))
x = ZeroPadding2D(padding=(1, 1))(x)

# Channel 1 - Convolution Net Layer 5
x = conv2D_lrn2d(x, 192, 13, 13, subsample=(1, 1))
x = ZeroPadding2D(padding=(1, 1))(x)

# Channel 1 - Cov Net Layer 6
x = conv2D_lrn2d(x, 128, 27, 27, subsample=(1, 1))
x = MaxPooling2D(
    strides=(2, 2), pool_size=(2, 2))(x)
x = ZeroPadding2D(padding=(1, 1))(x)

# Channel 1 - Cov Net Layer 7
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(DROPOUT)(x)

# Channel 1 - Cov Net Layer 8
x = Dense(2048, activation='relu')(x)
x = Dropout(DROPOUT)(x)

# Final Channel - Cov Net 9
model_output = Dense(output_dim=num_labels, activation='softmax')(x)

# return x, img_input, CONCAT_AXIS, INP_SHAPE, DIM_ORDERING
model = Model(input=img_input, output = model_output)


/storage/home1/shikhar_csa/try/my_proj/local/lib/python3.4/site-packages/ipykernel/__main__.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (11, 11), padding="same", kernel_regularizer=<keras.reg..., use_bias=False, bias_regularizer=<keras.reg..., strides=(1, 1), activation="relu")`
/storage/home1/shikhar_csa/try/my_proj/local/lib/python3.4/site-packages/ipykernel/__main__.py:36: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/storage/home1/shikhar_csa/try/my_proj/local/lib/python3.4/site-packages/ipykernel/__main__.py:41: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/storage/home1/shikhar_csa/try/my_proj/local/lib/python3.4/site-packages/ipykernel/__main__.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (55, 55), padding="same", kernel_regularizer=<keras.reg..., use_b

In [12]:
import time
import pylab as pl
from IPython import display
pl.style.use('ggplot')
%matplotlib inline

class Histories(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.acc = []
        self.loss = []
        self.val_loss = []
        self.val_acc = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs['acc'])
        self.loss.append(logs['loss'])
#         self.val_acc.append(logs['val_acc'])
#         self.val_loss.append(logs['val_loss'])
        
        pl.hold(True)
        plt.rcParams["figure.figsize"] = (8,5)
        pl.plot(self.acc, 'r')
        pl.plot(self.loss, 'b')
        pl.plot(self.val_acc, 'g')
        pl.plot(self.val_loss, 'k')
        pl.legend(['Train acc','Train loss','Valid acc', 'Valid loss'], loc=2)
        display.clear_output(wait=True)
        display.display(pl.gcf())
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [13]:
model.compile(optimizer=optimizers.adam(), loss='categorical_crossentropy', metrics=["accuracy"])

# histories = Histories()
# res = model.fit(x_train, y_train,
#           batch_size = 50,
#           epochs=300, verbose=2, callbacks=[histories])

res = model.fit(x_train, y_train,
          batch_size = 50,
          epochs=300, verbose=1)

Epoch 1/300
1735/1735 [==============================] - 25s - loss: 14.9954 - acc: 0.0841        

KeyboardInterrupt: 

In [ ]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))